In [2]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.utils.data import Dataset
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import DataLoader

In [3]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3070


In [4]:
class wineDataset(Dataset):
    def __init__(self, csv_path):
        super().__init__()
        # Load data to pandas DataFrame
        df = pd.read_csv(csv_path)
        # Convert data to a NumPy array and assign to self.data
        self.data = df.to_numpy()
        
    # Implement __len__ to return the number of data samples
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        features = self.data[idx]
        # Assign last data column to label
        return features

In [11]:
df1 = pd.read_csv('data/X_train.csv')
df2 = pd.read_csv('data/y_train.csv')
df = pd.concat([df1, df2], axis=1)
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.8,0.500,0.17,1.6,0.082,21.0,102.0,0.99600,3.39,0.48,9.5,5
1,7.6,0.290,0.49,2.7,0.092,25.0,60.0,0.99710,3.31,0.61,10.1,6
2,6.9,0.400,0.24,2.5,0.083,30.0,45.0,0.99590,3.26,0.58,10.0,5
3,8.9,0.635,0.37,1.7,0.263,5.0,62.0,0.99710,3.00,1.09,9.3,5
4,9.4,0.300,0.56,2.8,0.080,6.0,17.0,0.99640,3.15,0.92,11.7,8
...,...,...,...,...,...,...,...,...,...,...,...,...
954,8.3,0.540,0.24,3.4,0.076,16.0,112.0,0.99760,3.27,0.61,9.4,5
955,5.8,0.610,0.11,1.8,0.066,18.0,28.0,0.99483,3.55,0.66,10.9,6
956,6.6,0.440,0.15,2.1,0.076,22.0,53.0,0.99570,3.32,0.62,9.3,5
957,6.4,0.570,0.14,3.9,0.070,27.0,73.0,0.99669,3.32,0.48,9.2,5


In [12]:
#merge two csv together
train_dataloader = DataLoader(wineDataset('data/wine.csv'), batch_size=512, shuffle=True)
#create batches of data for training
real_batch = next(iter(train_dataloader))

In [13]:
real_batch[0]

tensor([ 6.7000,  1.0400,  0.0800,  2.3000,  0.0670, 19.0000, 32.0000,  0.9965,
         3.5200,  0.5700, 11.0000,  4.0000], dtype=torch.float64)

In [14]:
class Generator(nn.Module):
    def __init__(self, in_dim):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            nn.Linear(in_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 12)
        )

    def forward(self, input):
        return self.gen(input)


In [15]:
netG = Generator(12).to(device)
print(netG)

Generator(
  (gen): Sequential(
    (0): Linear(in_features=12, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=12, bias=True)
  )
)


In [17]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.dis = nn.Sequential(
            nn.Linear(12, 24),
            nn.ReLU(),
            nn.Linear(24, 36),
            nn.ReLU(),
            nn.Linear(36, 1),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.dis(input)

In [18]:
netD = Discriminator().to(device)

In [19]:
criterion = nn.BCELoss()

real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=0.0001, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.0001, betas=(0.5, 0.999))

In [20]:
real_label = torch.ones((1), dtype=torch.float32, device=device)
fake_label = torch.zeros((1), dtype=torch.float32, device=device)

In [21]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0
num_epochs = 500
print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(train_dataloader, 0):
        ############################
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device).float()
        b_size = real_cpu.size(0)

        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        # Calculate loss on all-real batch
    
        errD_real = criterion(output, real_label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, 1, 1, device=device)
        noise = torch.transpose(noise, 1, 2)
        # Generate fake image batch with G
        fake = netG(noise.squeeze())
        
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, fake_label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, real_label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()
        print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(train_dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(train_dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

Starting Training Loop...
[0/500][0/4]	Loss_D: 1.3630	Loss_G: 0.5875	D(x): 0.5760	D(G(z)): 0.5558 / 0.5557
[0/500][0/4]	Loss_D: 1.3630	Loss_G: 0.5875	D(x): 0.5760	D(G(z)): 0.5558 / 0.5557
[0/500][1/4]	Loss_D: 1.5590	Loss_G: 0.5888	D(x): 0.4728	D(G(z)): 0.5550 / 0.5550
[0/500][2/4]	Loss_D: 1.4550	Loss_G: 0.5893	D(x): 0.5242	D(G(z)): 0.5548 / 0.5547
[0/500][3/4]	Loss_D: 1.6317	Loss_G: 0.5869	D(x): 0.4406	D(G(z)): 0.5561 / 0.5560
[1/500][0/4]	Loss_D: 1.4791	Loss_G: 0.5848	D(x): 0.5146	D(G(z)): 0.5572 / 0.5572
[1/500][0/4]	Loss_D: 1.4791	Loss_G: 0.5848	D(x): 0.5146	D(G(z)): 0.5572 / 0.5572
[1/500][1/4]	Loss_D: 1.3895	Loss_G: 0.5873	D(x): 0.5611	D(G(z)): 0.5559 / 0.5558
[1/500][2/4]	Loss_D: 1.4198	Loss_G: 0.5844	D(x): 0.5464	D(G(z)): 0.5575 / 0.5574
[1/500][3/4]	Loss_D: 1.3527	Loss_G: 0.5802	D(x): 0.5875	D(G(z)): 0.5599 / 0.5598
[2/500][0/4]	Loss_D: 1.3511	Loss_G: 0.5874	D(x): 0.5830	D(G(z)): 0.5558 / 0.5557
[2/500][0/4]	Loss_D: 1.3511	Loss_G: 0.5874	D(x): 0.5830	D(G(z)): 0.5558 / 0.5557
[2

In [23]:
noise = torch.randn(12, 1, 1, device=device)
noise = torch.transpose(noise, 1, 2)
# Generate fake image batch with G
with torch.no_grad():
    fake = netG(noise.squeeze())
    print(fake)

tensor([ 2.1682, -0.0491,  1.0265,  1.3916, -0.1369,  3.7339,  9.6471, -0.4360,
         2.3866, -0.3670,  2.7037,  1.2630], device='cuda:0')
